# Julian's Small Example

Julian came up with a small $n = 2$ example (excluding sex data) for testing the problem. That's solved in this notebook, which requires the following imports.

In [1]:
import numpy as np                  # defines matrix structures
from qpsolvers import solve_qp      # used for quadratic optimization
from time import perf_counter       # fine grained timing 
import gurobipy as gp               # Gurobi optimization interface (1)
from gurobipy import GRB            # Gurobi optimization interface (2)

# want to round rather than truncate when printing
np.set_printoptions(threshold=np.inf)

# only show numpy output to five decimal places
np.set_printoptions(formatter={'float_kind':"{:.8f}".format})

## Problem Variables

The variables for Julian's $n = 2$ problem are as follows:
$$
    \Sigma = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix},\quad\
    \bar{\mu} = \begin{bmatrix} 1 \\ 2 \end{bmatrix},\quad\
    \Omega = \begin{bmatrix} \frac{1}{9} & 0 \\ 0 & 4 \end{bmatrix}.
$$
These are stored in `A02.txt`, `EBV02.txt`, and `S02.txt` respectively, where the first and last are matrices in coordinate format. Note that this particular problem does not contain any sex data or weight bound data. The problem is read into Python using the following function.

In [2]:
def load_problem(A_filename, E_filename, S_filename, dimension=False):
    """
    Function which loads genetic selection problems in from a
    series of three data files.
    """

    def load_symmetric_matrix(filename, dimension):
        """
        Since NumPy doesn't have a stock way to load matrices
        stored in coordinate format format, this adds one.
        """

        matrix = np.zeros([dimension, dimension])

        with open(filename, 'r') as file:
            for line in file:
                i, j, entry = line.split(" ")
                # data files indexed from 1, not 0
                matrix[int(i)-1, int(j)-1] = entry
                matrix[int(j)-1, int(i)-1] = entry

        return matrix


    # if dimension wasn't supplied, need to find that
    if not dimension:
        # get dimension from EBV, since it's the smallest file
        with open(E_filename, 'r') as file:
            dimension = sum(1 for _ in file)

    # EBV isn't in coordinate format so can be loaded directly
    E = np.loadtxt(E_filename)  
    # A and S are stored by coordinates so need special loader
    A = load_symmetric_matrix(A_filename, dimension)
    S = load_symmetric_matrix(S_filename, dimension)

    return A, E, S, dimension

The standard problem with $\lambda = 0.5$ can then be solved using Gurobi as follows:

In [3]:
sigma, mubar, omega, n = load_problem(
    "A02.txt",
    "EBV02.txt",
    "S02.txt",
    2)

lam = 0.5

# initialise standard genetic selection model
model_std = gp.Model("n02standard")
w_std = model_std.addMVar(shape=n, vtype=GRB.CONTINUOUS, name="w")

# define the objective functions for standard problem
model_std.setObjective(
    w_std.transpose()@mubar - (lam/2)*w_std.transpose()@(sigma@w_std),
GRB.MAXIMIZE)

# add sum-to-one constraint
model_std.addConstr(np.ones([1,2]) @ w_std == 1, name="sum-to-one")

# solve problem with Gurobi
model_std.optimize()
print(f"Standard: {w_std.X}")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-26
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xd1ce5e64
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  QObjective range [5e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal  

And the robust problem with $\kappa = 1.0$ can be solved as follows:

In [4]:
# other than kappa, reuse problem variables from the last cell
kappa = 1.0

# initialise robust genetic selection model
model_rbs = gp.Model("n02robust")

# initialise w for both models, z for robust model 
w_rbs = model_rbs.addMVar(shape=n, vtype=GRB.CONTINUOUS, name="w")
z_rbs = model_rbs.addVar(name="z")

model_rbs.setObjective(
    w_rbs.transpose()@mubar - (lam/2)*w_rbs.transpose()@(sigma@w_rbs) - kappa*z_rbs,
GRB.MAXIMIZE)

# add sum-to-half constraints to both models
model_rbs.addConstr(np.ones([1,2]) @ w_std == 1, name="sum-to-one")
# add quadratic uncertainty constraint to the robust model
model_rbs.addConstr(z_rbs**2 >= np.inner(w_rbs, omega@w_rbs), name="uncertainty")
model_rbs.addConstr(z_rbs >= 0, name="z positive")

# solve problem with Gurobi, print alongside standard solution for comparison
model_rbs.optimize()
print(f"Standard: {w_std.X}")
print(f"Robust:   {w_rbs.X}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x41322026
Model has 2 quadratic objective terms
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-01, 4e+00]
  Objective range  [1e+00, 2e+00]
  QObjective range [5e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.04s
Presolved: 7 rows, 8 columns, 12 nonzeros
Presolved model has 3 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.700e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual

Note that if $\kappa = 0$, the standard solution not _exactly_ recovered, only accurate to eight decimal places.

In [5]:
# other than kappa, reuse problem variables from the last cell
kappa = 0.0

# initialise robust genetic selection model
model_rbs = gp.Model("n02robust_k0")

# initialise w for both models, z for robust model 
w_rbs = model_rbs.addMVar(shape=n, vtype=GRB.CONTINUOUS, name="w")
z_rbs = model_rbs.addVar(name="z")

model_rbs.setObjective(
    w_rbs.transpose()@mubar - (lam/2)*w_rbs.transpose()@(sigma@w_rbs) - kappa*z_rbs,
GRB.MAXIMIZE)

# add sum-to-half constraints to both models
model_rbs.addConstr(np.ones([1,2]) @ w_std == 1, name="sum-to-one")
# add quadratic uncertainty constraint to the robust model
model_rbs.addConstr(z_rbs**2 >= np.inner(w_rbs, omega@w_rbs), name="uncertainty")
model_rbs.addConstr(z_rbs >= 0, name="z positive")

# solve problem with Gurobi, print alongside standard solution for comparison
model_rbs.optimize()
np.set_printoptions(formatter={'float_kind':"{:.10f}".format})
print(f"Standard: {w_std.X}")
print(f"Robust:   {w_rbs.X}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xe31c34b4
Model has 2 quadratic objective terms
Model has 2 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-01, 4e+00]
  Objective range  [1e+00, 2e+00]
  QObjective range [5e-01, 5e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.02s
Presolved: 7 rows, 8 columns, 12 nonzeros
Presolved model has 3 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.700e+01
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual